# Installations

In [ ]:
!pip3 install box2d-py
!pip3 install gym[Box_2D]

     |████████████████████████████████| 448 kB 5.2 MB/s 


In [ ]:
import os
import Box2D
import pyglet
import imageio
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
import gym
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

# Test Env

In [ ]:
env = gym.make("LunarLanderContinuous-v2")

In [ ]:
env.reset()

array([ 0.00551939,  1.4204907 ,  0.559039  ,  0.42535397, -0.00638881,
       -0.12663053,  0.        ,  0.        ], dtype=float32)

In [ ]:
env.action_space, env.observation_space

(Box(-1.0, 1.0, (2,), float32), Box(-inf, inf, (8,), float32))

In [ ]:
for episode in range(1): 
    observation = env.reset()
    for step in range(1):
        action = env.action_space.sample()  # or given a custom model, action = policy(observation)
        observation, reward, done, info = env.step(action)
        print(observation, reward, done, info, action)

[ 0.00684528  1.4323684   0.3461917   0.4637565  -0.00783924 -0.0776073
  0.          0.        ] 0.6751802091694685 False {} [-0.97176886  0.20109253]


# First skeleton

In [ ]:
class ActorCritic(nn.Module):

  def __init__(self, state_dim, action_dim, std_init):
    super(ActorCritic).__init__()

    # TBD switch to variable std

    self.critic = nn.Sequential(
        nn.Linear(state_dim, 64),
        nn.Tanh(),
        nn.Linear(64, 64),
        nn.Tanh(),
        nn.Linear(64, 1)
    )

    self.actor = nn.Sequential(
        nn.Linear(state_dim, 64),
        nn.Tanh(),
        nn.Linear(64, 64),
        nn.Tanh(),
        nn.Linear(64, action_dim),
        nn.Tanh()
    )

    self.action_vars = torch.full((self.action_dim,), std_init * std_init)
    self.cov_mat = torch.diag(self.action_vars).unsqueeze(dim=0)  # do we need the unsqueeze? 


  def get_policy(self, state):
    means = self.actor(state)
    policy = torch.distributions.MultivariateNormal(means, std)



## covariance stuff

Src: https://github.com/nikhilbarhate99/PPO-PyTorch/blob/master/PPO_colab.ipynb

In [ ]:
torch.full((2,), 3 * 3)

tensor([9, 9])

In [ ]:
torch.diag(torch.full((2,), 3 * 3)).unsqueeze(dim=0)

tensor([[[9, 0],
         [0, 9]]])